In [15]:
# alibi
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from alibi_detect.cd import KSDrift
from alibi_detect.models.tensorflow import scale_by_instance
from alibi_detect.utils.fetching import fetch_tf_model, fetch_detector
from alibi_detect.saving import save_detector, load_detector

# wilds
from typing import Tuple, Generator, Callable, Optional
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds import get_dataset
import alibi_detect
import torchvision
import wilds
from alibi_detect.models.pytorch import trainer

print(f"Alibi Detect Version: {alibi_detect.__version__}")
print(f"Torch Version: {torch.__version__}")
print(f"Torchvision Version: {torchvision.__version__}")
print(f"Wilds Version: {wilds.__version__}")


torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cpu')
%matplotlib inline


Alibi Detect Version: 0.10.4
Torch Version: 1.13.0+cu117
Torchvision Version: 0.14.0+cu117
Wilds Version: 2.0.0


In [16]:
# datensatz camelyon laden
%cd  '/home/jinglewsl/evoila/projects/image-drift-monitoring'
%pwd
camelyon = get_dataset(dataset="camelyon17",download=False)

WILDS_PATH = 'data'
N = 5000 # Size of reference set
DOWNLOAD = False
img_size = (96,96)
ds_train = camelyon.get_subset('train',transform=transforms.Compose([transforms.Resize(img_size), transforms.ToTensor()]))
train_loader = iter(get_train_loader('standard',ds_train,1))

ds_test = camelyon.get_subset('test',transform=transforms.Compose([transforms.Resize(img_size), transforms.ToTensor()]))
# test_loader = iter(get_train_loader('standard',ds_test,1))

x_ref = np.stack([next(train_loader)[0][0].numpy() for _ in range(N)],axis=0)
# x_test = np.stack([next(test_loader)[0][0].numpy() for _ in range(N)],axis=0)

train_loader_train = get_train_loader('standard',ds_train,1)

# x_ref = np.stack(data,axis=0)
print('here')
tmp = np.stack([tensor[0][0].numpy() for tensor ,i in zip(train_loader_train,range(N)) if i <N],axis=0)
# for labeled_batch in train_loader_train:
#     test = labeled_batch[0][0].numpy() 
#     tmp = np.stack(test,axis=0)    

#     i +=1
#     if i >=1:
        # break
print(x_ref.shape)
print(tmp.shape)
print(type(x_ref))

/home/jinglewsl/evoila/projects/image-drift-monitoring
here
(5000, 3, 96, 96)
(5000, 3, 96, 96)
<class 'numpy.ndarray'>


In [17]:
# We split the original test set in a reference dataset and a dataset which should not be rejected under the H0 of the K-S test.
# We also split the corrupted data by corruption type:
np.random.seed(0)
n_test = x_ref.shape[0]
idx = np.random.choice(n_test, size=n_test // 2, replace=False)
idx_h0 = np.delete(np.arange(n_test), idx, axis=0)
X_ref = x_ref[idx]
X_h0= x_ref[idx_h0]
print(X_ref.shape, X_h0.shape)
# print(x_ref.shape, x_test.shape)

(2500, 3, 96, 96) (2500, 3, 96, 96)


In [18]:
# Random encoder (Untrained Auto Encoder)
from functools import partial
from tensorflow.keras.layers import Conv2D, Dense, Flatten, InputLayer, Reshape
from alibi_detect.cd.tensorflow import preprocess_drift

tf.random.set_seed(0)

# define encoder
encoding_dim = 96
encoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(3,96,96)),
      Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(256, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(encoding_dim,)
  ]
)

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=encoder_net, batch_size=512)


# initialise drift detector
p_val = .05
cd = KSDrift(X_ref, p_val=p_val, preprocess_fn=preprocess_fn)

# we can also save/load an initialised detector
filepath = '/home/jinglewsl/evoila/projects/image-drift-monitoring/src/modules/alibi_detect/pca_ks_detector_camelyon'  # change to directory where detector is saved
save_detector(cd, filepath)

In [19]:
assert cd.p_val / cd.n_features == p_val / encoding_dim

In [20]:
#Let’s check whether the detector thinks drift occurred on the different test sets and time the prediction calls:

from timeit import default_timer as timer

labels = ['No!', 'Yes!']

def make_predictions(cd, x_h0,x_corr=None):
    t = timer()
    preds = cd.predict(x_h0)
    dt = timer() - t
    print('No corruption')
    print('Drift? {}'.format(labels[preds['data']['is_drift']]))
    print('Feature-wise p-values:')
    print(preds['data']['p_val'])
    print('len:{}'.format(len(preds['data']['p_val'])))
    print(f'Time (s) {dt:.3f}')


    if len(x_corr) > 0:
        t = timer()
        preds_2 = cd.predict(x_corr)
        dt = timer() - t
        print('')
        print('Drift? {}'.format(labels[preds_2['data']['is_drift']]))
        print('Feature-wise p-values:')
        print(preds_2['data']['p_val'])
        print('len:{}'.format(len(preds_2['data']['p_val'])))
        print(f'Time (s) {dt:.3f}')
        return preds_2  
    

In [21]:
# We split the original test set in a reference dataset and a dataset which should not be rejected under the H0 of the K-S test.
# We also split the corrupted data by corruption type:
np.random.seed(0)
n_test = x_test.shape[0]
idx = np.random.choice(n_test, size=2, replace=False)
idx_h0 = np.delete(np.arange(n_test), idx, axis=0)



x_test_subset = x_test[idx] 

print(len(x_test))
print(len(x_test_subset))

NameError: name 'x_test' is not defined

In [ ]:
make_predictions(cd, X_ref,x_test)

In [ ]:
preds_2 =make_predictions(cd,x_ref,x_test_subset)

In [ ]:
p_above_threshold = []
p_below_threshold = []
for p_val_pred in preds_2['data']['p_val']:
    if p_val_pred < p_val:
        p_below_threshold.append(p_val_pred)
    else:
        p_above_threshold.append(p_val_pred)

In [ ]:
len(p_above_threshold)

In [ ]:
len(p_below_threshold)

In [ ]:
fig, axis = plt.subplots(nrows=1,ncols=6,figsize=(15,4))
for i in range(6):
    axis[i].imshow(np.transpose(x_ref[i],(1,2,0)))
    axis[i].axis('off')

In [ ]:
fig, axis = plt.subplots(nrows=1,ncols=6,figsize=(15,4))
for i in range(6):
    axis[i].imshow(np.transpose(x_test[i],(1,2,0)))
    axis[i].axis('off')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

in_array = [0, math.pi / 2, np.pi / 3, np.pi]
out_array = np.tanh(in_array)

x = np.linspace(-10,10,100)
z = 1/(1 + np.exp(-x))
plt.plot(x,z)
plt.xlabel('x')
plt.ylabel("Sigmoid(X)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def sigmoid(x):
    s=1/(1+np.exp(-x))
    ds=s*(1-s)  
    return s,ds
x=np.arange(-6,6,0.01)
sigmoid(x)
# Setup centered axes
fig, ax = plt.subplots(figsize=(9, 5))
ax.spines['left'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
# Create and show plot
ax.plot(x,sigmoid(x)[0], color="#307EC7", linewidth=3, label="sigmoid")
ax.plot(x,sigmoid(x)[1], color="#9621E2", linewidth=3, label="derivative")
ax.legend(loc="upper right", frameon=False)
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def sigmoid(x):
    s=1/(1+np.exp(-x))
    ds=s*(1-s)  
    return s,ds
x=np.arange(-4,4,0.01)
sigmoid(x)


def relu(x):
 return np.maximum(0, x)
def tanh(x):
    t=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    dt=1-t**2
    return t,dt
z=np.arange(-4,4,0.01)
tanh(z)[0].size,tanh(z)[1].size
# Setup centered axes
fig, ax = plt.subplots(figsize=(9, 5))
# ax.spines['left'].set_position('center')
# ax.spines['bottom'].set_position('center')
# ax.spines['right'].set_color('none')
# ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
# Create and show plot
ax.plot(z,tanh(z)[0], color="#307EC7", linewidth=3, label="tanh")
ax.plot(x,sigmoid(x)[0], color="#9621E2", linewidth=3, label="sigmoid")
# ax.plot(z,tanh(z)[1], color="#9621E2", linewidth=3, label="derivative")
ax.plot(z,relu(z),color="#000000",linewidth=3, label="ReLU")
ax.legend(loc="upper left", frameon=False)
fig.show()